In [1]:
# Codeblock 1
import torch
import torch.nn as nn

In [4]:
# Codeblock 2
class ConvBlock(nn.Module):
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 kernel_size, 
                 padding):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels,
                              out_channels=out_channels, 
                              kernel_size=kernel_size, 
                              padding=padding)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.1)    #(1)
        
    def forward(self, x):
        #print(f'original\t: {x.size()}')

        x = self.conv(x)
        #print(f'after conv\t: {x.size()}')
        
        x = self.leaky_relu(x)
        #print(f'after leaky relu: {x.size()}')
        
        return x

In [3]:
# Codeblock 3
convblock = ConvBlock(in_channels=3,
                      out_channels=64,
                      kernel_size=3,
                      padding=1)
x = torch.randn(1, 3, 416, 416)
out = convblock(x)

original	: torch.Size([1, 3, 416, 416])
after conv	: torch.Size([1, 64, 416, 416])
after leaky relu: torch.Size([1, 64, 416, 416])


In [5]:
# Codeblock 4a
class Darknet(nn.Module):
    def __init__(self):
        super(Darknet, self).__init__()
        
        
        self.stage0 = nn.ModuleList([
            ConvBlock(3, 32, 3, 1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
        
        self.stage1 = nn.ModuleList([
            ConvBlock(32, 64, 3, 1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
            
        self.stage2 = nn.ModuleList([
            ConvBlock(64, 128, 3, 1), 
            ConvBlock(128, 64, 1, 0), 
            ConvBlock(64, 128, 3, 1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
        
        self.stage3 = nn.ModuleList([
            ConvBlock(128, 256, 3, 1), 
            ConvBlock(256, 128, 1, 0), 
            ConvBlock(128, 256, 3, 1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        ])
        
        self.stage4 = nn.ModuleList([
            ConvBlock(256, 512, 3, 1), 
            ConvBlock(512, 256, 1, 0), 
            ConvBlock(256, 512, 3, 1), 
            ConvBlock(512, 256, 1, 0), 
            ConvBlock(256, 512, 3, 1), 
        ])
        
        self.stage5 = nn.ModuleList([
            nn.MaxPool2d(kernel_size=2, stride=2),    #(1)
            ConvBlock(512, 1024, 3, 1), 
            ConvBlock(1024, 512, 1, 0), 
            ConvBlock(512, 1024, 3, 1), 
            ConvBlock(1024, 512, 1, 0), 
            ConvBlock(512, 1024, 3, 1), 
        ])
        
# Codeblock 4b
    def forward(self, x):
        print(f'original\t: {x.size()}')
        
        print()
        for i in range(len(self.stage0)):
            x = self.stage0[i](x)
            print(f'after stage0 #{i}\t: {x.size()}')
        
        print()
        for i in range(len(self.stage1)):
            x = self.stage1[i](x)
            print(f'after stage1 #{i}\t: {x.size()}')
        
        print()
        for i in range(len(self.stage2)):
            x = self.stage2[i](x)
            print(f'after stage2 #{i}\t: {x.size()}')
        
        print()
        for i in range(len(self.stage3)):
            x = self.stage3[i](x)
            print(f'after stage3 #{i}\t: {x.size()}')
        
        print()
        for i in range(len(self.stage4)):
            x = self.stage4[i](x)
            print(f'after stage4 #{i}\t: {x.size()}')
            
        x_stage4 = x.clone()        #(1)
        
        print()
        for i in range(len(self.stage5)):
            x = self.stage5[i](x)
            print(f'after stage5 #{i}\t: {x.size()}')
        
        x_stage5 = x.clone()        #(2)

        return x_stage4, x_stage5   #(3)        

In [6]:
# Codeblock 5
darknet = Darknet()

x = torch.randn(1, 3, 416, 416)
out = darknet(x)

original	: torch.Size([1, 3, 416, 416])

after stage0 #0	: torch.Size([1, 32, 416, 416])
after stage0 #1	: torch.Size([1, 32, 208, 208])

after stage1 #0	: torch.Size([1, 64, 208, 208])
after stage1 #1	: torch.Size([1, 64, 104, 104])

after stage2 #0	: torch.Size([1, 128, 104, 104])
after stage2 #1	: torch.Size([1, 64, 104, 104])
after stage2 #2	: torch.Size([1, 128, 104, 104])
after stage2 #3	: torch.Size([1, 128, 52, 52])

after stage3 #0	: torch.Size([1, 256, 52, 52])
after stage3 #1	: torch.Size([1, 128, 52, 52])
after stage3 #2	: torch.Size([1, 256, 52, 52])
after stage3 #3	: torch.Size([1, 256, 26, 26])

after stage4 #0	: torch.Size([1, 512, 26, 26])
after stage4 #1	: torch.Size([1, 256, 26, 26])
after stage4 #2	: torch.Size([1, 512, 26, 26])
after stage4 #3	: torch.Size([1, 256, 26, 26])
after stage4 #4	: torch.Size([1, 512, 26, 26])

after stage5 #0	: torch.Size([1, 512, 13, 13])
after stage5 #1	: torch.Size([1, 1024, 13, 13])
after stage5 #2	: torch.Size([1, 512, 13, 13])
afte

In [7]:
# Codeblock 6
NUM_ANCHORS = 5
NUM_CLASSES = 20

In [8]:
# Codeblock 7a
class YOLOv2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.darknet = Darknet()                       #(1)
        
        self.passthrough = ConvBlock(512, 64, 1, 0)    #(2)
        
        self.stage6 = nn.ModuleList([                  #(3)
            ConvBlock(1024, 1024, 3, 1), 
            ConvBlock(1024, 1024, 3, 1), 
        ])

        self.stage7 = nn.ModuleList([
            ConvBlock(1280, 1024, 3, 1),
            ConvBlock(1024, NUM_ANCHORS*(5+NUM_CLASSES), 1, 0)    #(4)
        ])
        
# Codeblock 7b
    def reorder(self, x, scale=2):                      # ([1, 64, 26, 26])
        B, C, H, W = x.shape
        h, w = H // scale, W // scale

        x = x.reshape(B, C, h, scale, w, scale)         # ([1, 64, 13, 2, 13, 2])     
        x = x.transpose(3, 4)                           # ([1, 64, 13, 13, 2, 2])

        x = x.reshape(B, C, h * w, scale * scale)       # ([1, 64, 169, 4])
        x = x.transpose(2, 3)                           # ([1, 64, 4, 169])

        x = x.reshape(B, C, scale * scale, h, w)        # ([1, 64, 4, 13, 13])
        x = x.transpose(1, 2)                           # ([1, 4, 64, 13, 13])

        x = x.reshape(B, scale * scale * C, h, w)       # ([1, 256, 13, 13])

        return x
    
# Codeblock 7c
    def forward(self, x):
        
        x_stage4, x_stage5 = self.darknet(x)              #(1)
        print(f'\nx_stage4\t\t\t: {x_stage4.size()}')
        print(f'x_stage5\t\t\t: {x_stage5.size()}')
        
        print()
        x = x_stage5
        for i in range(len(self.stage6)):
            x = self.stage6[i](x)                         #(2)
            print(f'x_stage5 after stage6 #{i}\t: {x.size()}')    
        
        x_stage4 = self.passthrough(x_stage4)             #(3)
        print(f'\nx_stage4 after passthrough\t: {x_stage4.size()}')
        
        x_stage4 = self.reorder(x_stage4)                 #(4)
        print(f'x_stage4 after reorder\t\t: {x_stage4.size()}')
        
        x = torch.cat([x_stage4, x], dim=1)               #(5)
        print(f'\nx after concatenate\t\t: {x.size()}')
        
        for i in range(len(self.stage7)):                 #(6)
            x = self.stage7[i](x)
            print(f'x after stage7 #{i}\t\t: {x.size()}')    
        
        return x

In [9]:
# Codeblock 8
yolov2 = YOLOv2()
x = torch.randn(1, 3, 416, 416)

out = yolov2(x)

original	: torch.Size([1, 3, 416, 416])

after stage0 #0	: torch.Size([1, 32, 416, 416])
after stage0 #1	: torch.Size([1, 32, 208, 208])

after stage1 #0	: torch.Size([1, 64, 208, 208])
after stage1 #1	: torch.Size([1, 64, 104, 104])

after stage2 #0	: torch.Size([1, 128, 104, 104])
after stage2 #1	: torch.Size([1, 64, 104, 104])
after stage2 #2	: torch.Size([1, 128, 104, 104])
after stage2 #3	: torch.Size([1, 128, 52, 52])

after stage3 #0	: torch.Size([1, 256, 52, 52])
after stage3 #1	: torch.Size([1, 128, 52, 52])
after stage3 #2	: torch.Size([1, 256, 52, 52])
after stage3 #3	: torch.Size([1, 256, 26, 26])

after stage4 #0	: torch.Size([1, 512, 26, 26])
after stage4 #1	: torch.Size([1, 256, 26, 26])
after stage4 #2	: torch.Size([1, 512, 26, 26])
after stage4 #3	: torch.Size([1, 256, 26, 26])
after stage4 #4	: torch.Size([1, 512, 26, 26])

after stage5 #0	: torch.Size([1, 512, 13, 13])
after stage5 #1	: torch.Size([1, 1024, 13, 13])
after stage5 #2	: torch.Size([1, 512, 13, 13])
afte